Create Azure SQL Virtual Machine
============================================

Steps of this procedure include:
1. Set variables and set up Notebook 
1. Connect to Azure account and subscription 
1. Configure Network Settings 
1. Provision virtual machine resource in Azure 
1. Provision SQL VM resource in Azure

## Set variables
These variables are set based on your inputs in the deployment wizard. You can make changes to these variables but be aware of possible validation errors caused by your changes.






## Notebook setup

In [ ]:
import sys, json, time, string, random, subprocess

if "AZDATA_NB_VAR_AZURE_SQLVM_PASSWORD" in os.environ:
    azure_sqlvm_password = os.environ["AZDATA_NB_VAR_AZURE_SQLVM_PASSWORD"]

if "AZDATA_NB_VAR_AZURE_SQLVM_SQL_PASSWORD" in os.environ:
    azure_sqlvm_sqlAuthenticationPassword = os.environ["AZDATA_NB_VAR_AZURE_SQLVM_SQL_PASSWORD"]

def run_command(command, json_decode = True, printOutput = True):
    print(command)
    process = subprocess.Popen(command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    output, error = process.communicate()
    if process.returncode != 0: 
        print("Process failed %d \n%s" % (process.returncode, error.decode("utf-8")))
        raise Exception()
    if output:
        output = output.decode("utf-8")
        if printOutput:
            print(output)
        try:
            return json.loads(output)
        except:
            return output

def get_random_string(length):
    letters = string.ascii_lowercase
    result_str = ''.join(random.choice(letters) for i in range(length))
    print("Random string of length", length, "is:", result_str)
    return result_str

## Connecting to your Azure account


In [ ]:
subscriptions = run_command('az account list', printOutput = False)
if azure_sqlvm_nb_var_subscription not in (subscription["id"] for subscription in subscriptions):
    run_command('az login')

## Setting your Azure subscription


In [ ]:
run_command(
    'az account set '
    '--subscription {0}'
    .format(
        azure_sqlvm_nb_var_subscription));

## Configure network settings
All networking configurations are handled in this step, including virtual network, subnet, public IP address, network security group, connectivity settings, and network interface.

1. If you selected the option to create a new virtual network, subnet, or a public IP address, they will be created here. These resources are used to provide network connectivity to the virtual machine and connect it to the internet.

In [ ]:
subnet_name = azure_sqlvm_subnet
vnet_name = azure_sqlvm_virtnet
pip_name = azure_sqlvm_publicip


if azure_sqlvm_newVirtualNetwork:
   run_command(
      'az network vnet create '
      '--resource-group {0} '
      '--name {1} '
      '--location {2} '
      '--address-prefixes 192.168.0.0/16 '
      .format(
         azure_sqlvm_nb_var_resource_group_name, 
         vnet_name, 
         azure_sqlvm_location));

if azure_sqlvm_newSubnet:
   run_command(
      'az network vnet subnet create '
      '--name {0} '
      '--resource-group {1} '
      '--vnet-name {2} '
      '--address-prefixes 192.168.1.0/24'
      .format(
         subnet_name,
         azure_sqlvm_nb_var_resource_group_name,
         vnet_name
         ));

if azure_sqlvm_newPublicIp:
   run_command(
      'az network public-ip create '
      '--resource-group {0} '
      '--location {1} '
      '--allocation-method Static '
      '--idle-timeout 4 '
      '--name {2}'
      .format(
         azure_sqlvm_nb_var_resource_group_name, 
         azure_sqlvm_location, 
         pip_name));

2. Create a network security group and configure rules to allow remote desktop (RDP) and SQL Server connections.

In [ ]:
nsg_name = azure_sqlvm_nb_var_resource_group_name + 'nsg'
nsg_name_id = run_command(
   'az network nsg create '
   '--resource-group {0} '
   '--location {1} '
   '--name {2} '
   .format(
      azure_sqlvm_nb_var_resource_group_name,
      azure_sqlvm_location, 
      nsg_name));

if azure_sqlvm_allow_rdp:
   run_command(
      'az network nsg rule create '
      '--name RDPRule '
      '--nsg-name {0} '
      '--priority 1000 '
      '--resource-group {1} '
      '--protocol Tcp '
      '--direction Inbound '
      '--source-address-prefixes * '
      '--source-port-range * '
      '--destination-address-prefixes * '
      '--destination-port-range 3389  '
      '--access Allow'
      .format(
         nsg_name,
         azure_sqlvm_nb_var_resource_group_name));

3. Create the network interface.

In [ ]:
interface_name = azure_sqlvm_nb_var_resource_group_name + "int"

command = (
   'az network nic create '
   '--name {0} '
   '--resource-group {1} '
   '--subnet {2} '
   '--public-ip-address {3} '
   '--network-security-group {4} '
   '--location {5} '
)

if azure_sqlvm_newSubnet:
   command += '--vnet-name {6} '

run_command(
   command
   .format(
      interface_name,
      azure_sqlvm_nb_var_resource_group_name,
      subnet_name,
      pip_name,
      nsg_name,
      azure_sqlvm_location,
      vnet_name));

## Create Azure Virtual Machine and Azure SQL VM resources
First the Azure VM will be created based on all the settings previously specified. Next the SQL VM will be created to include a default set of SQL connectivity settings. The SQL VM resource is where you can manage any SQL Server manageability features offered by Azure. [Learn more](https://docs.microsoft.com/azure/azure-sql/virtual-machines/windows/sql-server-on-azure-vm-iaas-what-is-overview) about what you can do with your Azure SQL VM after it has been created.

In [ ]:
# Create the VM
azure_sqlvm_image = 'sql2019-ws2019'

run_command(
   'az vm create '
   '--name {0} '
   '--size {1} '
   '--computer-name {0} '
   '--admin-username {2} '
   '--admin-password {3} '
   '--image {4}:{5}:{6}:{7} '
   '--nics {8} '
   '--resource-group {9} '
   '--location {10} '
   .format(
       azure_sqlvm_vmname,
       azure_sqlvm_vmsize,
       azure_sqlvm_username,
       azure_sqlvm_password,
       'MicrosoftSQLServer',
       azure_sqlvm_image,
       azure_sqlvm_image_sku,
       'latest',
       interface_name,
       azure_sqlvm_nb_var_resource_group_name,
       azure_sqlvm_location
   )
);

In [ ]:
command = (
    'az sql vm create '
    '--name {0} '
    '--license-type PAYG '
    '--resource-group {1} '
    '--connectivity-type {2} '
    '--sql-mgmt-type Full '
    '--location {3} '
)

if azure_sqlvm_enableSqlAuthentication:
    command += '--sql-auth-update-username {4} '
    command += '--sql-auth-update-pwd {5} '

if azure_sqlvm_sqlConnectivityType != 'local':
    command += '--port {6} '

run_command(
    command
    .format(
        azure_sqlvm_vmname,
        azure_sqlvm_nb_var_resource_group_name,
        azure_sqlvm_sqlConnectivityType,
        azure_sqlvm_location,
        azure_sqlvm_sqlAuthenticationUsername,
        azure_sqlvm_sqlAuthenticationPassword,
        azure_sqlvm_port,
    )
);